In [ ]:
import pandas as pd
import polars as pl
import numpy  as np
import networkx as nx
from math import sin, cos, pi, sqrt, atan2
import random
import time
import copy
import os
from rtsvg import *
rt = RACETrack()
#pts = [(20,20),(100,20),(100,100), (110,100), (300,100), (310,210), (310,300), (320,300), (330,300), (340,290), (350,280), (350, 50), (360,40), (370,30), (380,40), (390,50), (400,400)]
#pts = [(100,100), (150,180), (200,200), (220,300), (230,400), (260,410)]
pts = []
for _angle_ in range(0,365,5):
    xr = 10*(random.random() - 0.5)
    yr = 10*(random.random() - 0.5)
    _x_, _y_ = 250 + (200+xr) * cos(_angle_ * pi / 180), 250 + (200+yr) * sin(_angle_ * pi / 180)
    pts.append((_x_, _y_))

In [ ]:
svg = ['<svg x="0" y="0" width="500" height="500">']
svg.append('<rect x="0" y="0" width="500" height="500" fill="#000000" />')
pw = rt.piecewiseCubicBSpline(pts)
_colors_       = rt.co_mgr.brewerColors('qualitative', n=10)
_stroke_width_ = 2.0
_gap_          = 2.0
for i in range(len(pw)-1):
    x0, y0, x1, y1 = pw[i][0], pw[i][1], pw[i+1][0], pw[i+1][1]
    dx, dy = x1 - x0, y1 - y0
    _len_  = sqrt(dx**2 + dy**2)
    if _len_ < 0.001: _len_ = 0.001
    dx,  dy  = dx/_len_, dy/_len_
    pdx, pdy = dy, -dx
    for j in range(10):
        _x0_, _y0_, _x1_, _y1_ = x0, y0, x1, y1
        if j > 0:
            if j%2 == 0:
                _x0_ += pdx * (_gap_ + _stroke_width_) * (j/2)
                _y0_ += pdy * (_gap_ + _stroke_width_) * (j/2)
                _x1_ += pdx * (_gap_ + _stroke_width_) * (j/2)
                _y1_ += pdy * (_gap_ + _stroke_width_) * (j/2)
            else:
                _x0_ -= pdx * (_gap_ + _stroke_width_) * (int(j/2)+1)
                _y0_ -= pdy * (_gap_ + _stroke_width_) * (int(j/2)+1)
                _x1_ -= pdx * (_gap_ + _stroke_width_) * (int(j/2)+1)
                _y1_ -= pdy * (_gap_ + _stroke_width_) * (int(j/2)+1)
        _co_ = _colors_[j]
        svg.append(f'<line x1="{_x0_}" y1="{_y0_}" x2="{_x1_}" y2="{_y1_}" stroke="{_co_}" stroke-width="{_stroke_width_}" />')
svg.append('</svg>')
rt.tile([''.join(svg)])

In [ ]:
#
# This came from screwing up the math... but it looks good :( :)
# -- it's because i re-used pdx, pdy .... which were set above... and are fixed...
#
svg = ['<svg x="0" y="0" width="500" height="500">']
svg.append('<rect x="0" y="0" width="500" height="500" fill="#000000" />')
pw = rt.piecewiseCubicBSpline(pts)
_colors_       = rt.co_mgr.brewerColors('qualitative', n=10)
_stroke_width_ = 4.0
_gap_          = 1.0
for j in range(10):
    _co_                 = _colors_[j]
    _x1_last_, _y1_last_ = None, None
    _path_               = []
    for i in range(len(pw)-1):
        x0_base, y0_base, x1_base, y1_base = pw[i][0], pw[i][1], pw[i+1][0], pw[i+1][1]
        dx_base, dy_base = x1_base - x0_base, y1_base - y0_base
        _len_  = sqrt(dx_base**2 + dy_base**2)
        if _len_ < 0.001: continue # _len_ = 0.001 # appears to be an error in the piecewiseCubicBSpline where zero length segments are created
        dx_base,  dy_base  = dx_base/_len_,  dy_base/_len_
        pdx_base, pdy_base = dy_base,       -dx_base
        _x0_, _y0_, _x1_, _y1_ = x0_base, y0_base, x1_base, y1_base
        if j > 0:
            if j%2 == 0:
                _x0_ += pdx * (_gap_ + _stroke_width_) * (j/2)
                _y0_ += pdy * (_gap_ + _stroke_width_) * (j/2)
                _x1_ += pdx * (_gap_ + _stroke_width_) * (j/2)
                _y1_ += pdy * (_gap_ + _stroke_width_) * (j/2)
            else:
                _x0_ -= pdx * (_gap_ + _stroke_width_) * (int(j/2)+1)
                _y0_ -= pdy * (_gap_ + _stroke_width_) * (int(j/2)+1)
                _x1_ -= pdx * (_gap_ + _stroke_width_) * (int(j/2)+1)
                _y1_ -= pdy * (_gap_ + _stroke_width_) * (int(j/2)+1)
        _dx_local_, _dy_local_ = _x1_ - _x0_, _y1_ - _y0_
        _len_  = sqrt(_dx_local_**2 + _dy_local_**2)
        if _len_ < 0.001: continue
        _dx_local_, _dy_local_ = _dx_local_/_len_, _dy_local_/_len_    
        if _x1_last_ is not None: 
            _path_.append(f'L {_x0_} {_y0_} ')
            _path_.append(f'L {_x1_} {_y1_} ')
        else: _path_.append(f'M {_x0_} {_y0_} ')
        _x1_last_, _y1_last_ = _x1_, _y1_
    _path_description_ = ''.join(_path_)
    svg.append(f'<path d="{_path_description_}" stroke="{_co_}" fill="none" stroke-width="{_stroke_width_}" />')
svg.append('</svg>')
rt.tile([''.join(svg)])

In [ ]:
svg = ['<svg x="0" y="0" width="500" height="500">']
svg.append('<rect x="0" y="0" width="500" height="500" fill="#000000" />')
pw = rt.piecewiseCubicBSpline(pts)
_colors_       = rt.co_mgr.brewerColors('qualitative', n=10)
_stroke_width_ =  3.0
_gap_          =  1.0

_lu_ = {'pt':[], 'path':[], 'dot':[]}
for i in range(len(_colors_)): 
    rt.co_mgr.str_to_color_lu[i]      = _colors_[i]
    rt.co_mgr.str_to_color_lu[str(i)] = _colors_[i]

for j in range(10):
    _co_                           = _colors_[j]
    _x1_base_last_, _y1_base_last_ = None, None
    _x1_last_,      _y1_last_      = None, None
    _path_               = []
    for i in range(len(pw)-1):
        x0_base, y0_base, x1_base, y1_base = pw[i][0], pw[i][1], pw[i+1][0], pw[i+1][1]
        dx_base, dy_base = x1_base - x0_base, y1_base - y0_base
        _len_  = sqrt(dx_base**2 + dy_base**2)
        if _len_ < 0.001: continue # _len_ = 0.001 # appears to be an error in the piecewiseCubicBSpline where zero length segments are created
        dx_base,  dy_base  = dx_base/_len_,  dy_base/_len_
        pdx_base, pdy_base = dy_base,       -dx_base
        _x0_, _y0_, _x1_, _y1_ = x0_base, y0_base, x1_base, y1_base
        if j > 0:
            if j%2 == 0:
                _x0_ += pdx_base * (_gap_ + _stroke_width_) * (j/2)
                _y0_ += pdy_base * (_gap_ + _stroke_width_) * (j/2)
                _x1_ += pdx_base * (_gap_ + _stroke_width_) * (j/2)
                _y1_ += pdy_base * (_gap_ + _stroke_width_) * (j/2)
            else:
                _x0_ -= pdx_base * (_gap_ + _stroke_width_) * (int(j/2)+1)
                _y0_ -= pdy_base * (_gap_ + _stroke_width_) * (int(j/2)+1)
                _x1_ -= pdx_base * (_gap_ + _stroke_width_) * (int(j/2)+1)
                _y1_ -= pdy_base * (_gap_ + _stroke_width_) * (int(j/2)+1)
        _dx_local_, _dy_local_ = _x1_ - _x0_, _y1_ - _y0_
        _len_  = sqrt(_dx_local_**2 + _dy_local_**2)
        if _len_ < 0.001: continue
        _dx_local_, _dy_local_ = _dx_local_/_len_, _dy_local_/_len_

        render = True
        if _x1_base_last_ is not None:
            _my_dx_, _my_dy_ = _x0_ - _x1_last_, _y0_ - _y1_last_
            _len_ = sqrt(_my_dx_**2 + _my_dy_**2)
            if _len_ < 0.001: _len_ = 0.001
            _my_dx_, _my_dy_ = _my_dx_/_len_, _my_dy_/_len_
            #_lu_['pt'].append(i), _lu_['path'].append(j), _lu_['dot'].append(_my_dx_*_my_dy_ - _dx_local_*_dy_local_)
            _lu_['pt'].append(i), _lu_['path'].append(j), _lu_['dot'].append(atan2(dy_base, dx_base) - atan2(_dy_local_, _dx_local_))

        if render:
            if len(_path_) == 0: 
                _path_.append(f'M {_x0_} {_y0_} ')
                _path_.append(f'L {_x1_} {_y1_} ')
            else:
                _path_.append(f'L {_x0_} {_y0_} ')
                _path_.append(f'L {_x1_} {_y1_} ')
        
        svg.append(f'<circle cx="{(_x0_+_x1_)/2}" cy="{(_y0_+_y1_)/2}" r="1" fill="{_co_}" />')

        _x1_base_last_, _y1_base_last_ = x1_base, y1_base
        _x1_last_,      _y1_last_      = _x1_,    _y1_
    _path_description_ = ''.join(_path_)
    svg.append(f'<path d="{_path_description_}" stroke="{_co_}" fill="none" stroke-width="{_stroke_width_}" stroke-linecap="butt" />')
svg.append('</svg>')
df_debug = pd.DataFrame(_lu_)
rt.tile([''.join(svg), 
         rt.xy(df_debug, x_field='pt', y_field='dot', color_by='path', dot_size='large', w=800, h=500), 
         rt.histogram(df_debug, bin_by='path', color_by='path', w=256, h=500)])

In [ ]:
svg = ['<svg x="0" y="0" width="500" height="500">']
svg.append('<rect x="0" y="0" width="500" height="500" fill="#000000" />')
pw       = rt.piecewiseCubicBSpline(pts)
pw_fixed = []
for i in range(len(pw)-1):
    if pw[i][0] == pw[i+1][0] and pw[i][1] == pw[i+1][1]: pass
    else:                                                 pw_fixed.append(pw[i])
pw = pw_fixed

vecs  = []
perps = []
_colors_       = rt.co_mgr.brewerColors('qualitative', n=10)
_stroke_width_ =  2.0
_gap_          =  1.0
for i in range(len(pw)-1):
    x0, y0, x1, y1 = pw[i][0], pw[i][1], pw[i+1][0], pw[i+1][1]
    dx, dy = x1 - x0, y1 - y0
    _len_  = sqrt(dx**2 + dy**2)
    if _len_ < 0.00001: _len_ = 0.00001
    dx,  dy  = dx/_len_, dy/_len_
    pdx, pdy = dy, -dx
    vecs.append((dx,dy)), perps.append((pdx,pdy))

def renorm(dx, dy):
    _len_ = sqrt(dx**2 + dy**2)
    if _len_ < 0.0001: _len_ = 0.0001
    return dx/_len_, dy/_len_

for i in range(1, len(pw)-2):
    x0, y0, x1, y1 = pw[i][0], pw[i][1], pw[i+1][0], pw[i+1][1]
    pdx_last, pdy_last, pdx, pdy, pdx_next, pdy_next = perps[i-1][0], perps[i-1][1], perps[i][0], perps[i][1], perps[i+1][0], perps[i+1][1]
    pdx0, pdy0 = renorm((pdx_last+pdx)/2.0, (pdy_last+pdy)/2.0)
    pdx1, pdy1 = renorm((pdx+pdx_next)/2.0, (pdy+pdy_next)/2.0)
    for j in range(1):
        if   j > 0 and j%2 == 0: d_i = int(j/2)
        elif j > 0:              d_i = -1 * (int(j/2) + 1)
        else:                    d_i = 0
        d = (_gap_ + _stroke_width_) * d_i 
        _x0_, _y0_, _x1_, _y1_, _xm_, _ym_ = x0 + pdx_last*d, y0 + pdy_last*d, x1 + pdx*d, y1 + pdy*d, (x0+x1)/2.0 + pdx_next*d, (y0+y1)/2.0 + pdy_next*d
        _co_ = _colors_[j]
        svg.append(f'<line x1="{_x0_}" y1="{_y0_}" x2="{_xm_}" y2="{_ym_}" stroke="{_co_}" stroke-width="{0.2}" />')
        svg.append(f'<line x1="{_xm_}" y1="{_ym_}" x2="{_x1_}" y2="{_y1_}" stroke="{_co_}" stroke-width="{0.2}" />')

        _roff_ = 0.1 * (i%4)
        svg.append(f'<line x1="{_x0_}" y1="{_y0_}" x2="{_x0_ + pdx0*10}" y2="{_y0_+pdy0*10}" stroke="#ff0000" stroke-width="0.1" />')
        svg.append(f'<circle cx="{_x0_}" cy="{_y0_}" r="{0.4+_roff_}" fill="none" stroke="#ff0000" stroke-width="0.02" />')
        svg.append(f'<line x1="{_xm_}" y1="{_ym_}" x2="{_xm_ + pdx*20}" y2="{_ym_+pdy*20}" stroke="#ffffff" stroke-width="0.05" />')
        svg.append(f'<line x1="{_x1_}" y1="{_y1_}" x2="{_x1_ + pdx1*10}" y2="{_y1_+pdy1*10}" stroke="#00ff00" stroke-width="0.1" />')
        svg.append(f'<circle cx="{_x1_}" cy="{_y1_}" r="{0.3+_roff_}" fill="none" stroke="#00ff00" stroke-width="0.05" />')

for i in range(len(pts)): svg.append(f'<circle cx="{pts[i][0]}" cy="{pts[i][1]}" r="1.0" fill="none" stroke="#b0b0b0" stroke-width="0.2" />')

svg.append('</svg>')
rt.tile([''.join(svg)])

In [ ]:
_lu_ = {'i':[], 'd':[]}
for i in range(len(pw)-1):
    dx, dy = pw[i+1][0] - pw[i][0], pw[i+1][1] - pw[i][1]
    _len_ = sqrt(dx**2 + dy**2)
    _lu_['i'].append(i), _lu_['d'].append(_len_)
rt.xy(pd.DataFrame(_lu_), x_field='i', y_field='d', dot_size='large', w=800, h=300)

In [ ]:
svg = ['<svg x="0" y="0" width="500" height="500">']
svg.append('<rect x="0" y="0" width="500" height="500" fill="#ffffff" />')
pw       = rt.piecewiseCubicBSpline(pts)
vecs  = []
perps = []
_colors_        = rt.co_mgr.brewerColors('qualitative', n=10)
_stroke_width_  =   4.0
_gap_           =   1.0
_num_of_lanes_  =   7 
for i in range(len(pw)-1):
    x0, y0, x1, y1 = pw[i][0], pw[i][1], pw[i+1][0], pw[i+1][1]
    dx, dy = x1 - x0, y1 - y0
    _len_  = sqrt(dx**2 + dy**2)
    if _len_ < 0.00001: _len_ = 0.00001
    dx,  dy  = dx/_len_, dy/_len_
    pdx, pdy = dy, -dx
    vecs.append((dx,dy)), perps.append((pdx,pdy))

def renorm(dx, dy):
    _len_ = sqrt(dx**2 + dy**2)
    if _len_ < 0.0001: _len_ = 0.0001
    return dx/_len_, dy/_len_

path_descs = {}
for i in range(_num_of_lanes_): path_descs[i] = []

_len_lu_ = {'point':[], 'path':[], 'len':[]}
for i in range(1, len(pw)-2):
    x0, y0, x1, y1 = pw[i][0], pw[i][1], pw[i+1][0], pw[i+1][1]
    pdx_last, pdy_last, pdx, pdy, pdx_next, pdy_next = perps[i-1][0], perps[i-1][1], perps[i][0], perps[i][1], perps[i+1][0], perps[i+1][1]
    pdx0, pdy0 = renorm((pdx_last+pdx)/2.0, (pdy_last+pdy)/2.0)
    pdx1, pdy1 = renorm((pdx+pdx_next)/2.0, (pdy+pdy_next)/2.0)
    _base_len_ = None
    for j in range(_num_of_lanes_):
        if   j > 0 and j%2 == 0: d_i = int(j/2)
        elif j > 0:              d_i = -1 * (int(j/2) + 1)
        else:                    d_i = 0
        d = (_gap_ + _stroke_width_) * d_i 
        _x0_, _y0_, _x1_, _y1_, _xm_, _ym_ = x0 + pdx_last*d, y0 + pdy_last*d, x1 + pdx*d, y1 + pdy*d, (x0+x1)/2.0 + pdx_next*d, (y0+y1)/2.0 + pdy_next*d
        _co_ = _colors_[j]

        _seg_len_ = sqrt((_x0_-_x1_)**2 + (_y0_-_y1_)**2)
        if rt.segmentsIntersect(((x0, y0), (_x0_,_y0_)), ((x1, y1), (_x1_, _y1_)))[0]: _seg_len_ = -_seg_len_

        _len_lu_['point'].append(i),   _len_lu_['path'].append(j), _len_lu_['len'].append(_seg_len_)
        if j == 0: _base_len_ = _seg_len_

        if _seg_len_ > 0.0:
            if len(path_descs[j]) == 0: path_descs[j].append(f" {_x0_} {_y0_} ")
            path_descs[j].append(f" {_xm_} {_ym_} ")
            path_descs[j].append(f" {_x1_} {_y1_} ")

for i in range(_num_of_lanes_):
    _path_ = 'M' + 'L'.join(path_descs[i])
    _co_   = _colors_[i]
    svg.append(f'<path d="{_path_}" stroke="{_co_}" stroke-width="{_stroke_width_}" stroke-end-cap="round" fill="none" />')

svg.append('</svg>')
rt.tile([''.join(svg),
         rt.xy(pl.DataFrame(_len_lu_), x_field='point', y_field='len', color_by='path', line_groupby_field='path', line_groupby_w=3.0, dot_size='medium', w=1000, h=500)])

In [ ]:
svg = ['<svg x="0" y="0" width="500" height="500">']
svg.append('<rect x="0" y="0" width="500" height="500" fill="#ffffff" />')
pw       = rt.piecewiseCubicBSpline(pts, beta=0.9) # at beta=1.0, you get the expandSegmentsIntoPiecewiseCurvedParts results (pretty much)
for i in range(len(pts)):
    svg.append(f'<circle cx="{pts[i][0]}" cy="{pts[i][1]}" r="3" fill="#000000" />')
for i in range(len(pw)-1):
    x0, y0, x1, y1 = pw[i][0], pw[i][1], pw[i+1][0], pw[i+1][1]
    svg.append(f'<line x1="{x0}" y1="{y0}" x2="{x1}" y2="{y1}" stroke="#000000" stroke-width="2" />')
pw = rt.expandSegmentsIntoPiecewiseCurvedParts(pts)
for i in range(len(pw)-1):
    x0, y0, x1, y1 = pw[i][0], pw[i][1], pw[i+1][0], pw[i+1][1]
    svg.append(f'<line x1="{x0}" y1="{y0}" x2="{x1}" y2="{y1}" stroke="#000000" stroke-width="2" />')
svg.append('</svg>')
rt.tile([''.join(svg)])